In [1]:
from pathlib import Path
from typing import List

from matplotlib import pyplot as plt
from matplotlib.axes import Axes

import vcsvgenie.waveform
from vcsvgenie import transient_waveform
from vcsvgenie.read import read_vcsv

In [2]:
SPECIFICATION = transient_waveform.TransientResultSpecification(
    inputs=[
        "/comp_a<3>", # these are the wire names in your DUT/testbench
        "/comp_a<2>", # (depending on what you test)
        "/comp_a<1>",
        "/comp_a<0>",
        "/comp_b<3>",
        "/comp_b<2>",
        "/comp_b<1>",
        "/comp_b<0>",
    ],
    outputs=[
        "/comp_z<7>", # they don't necessarily have to be in order
        "/comp_z<6>",
        "/comp_z<5>",
        "/comp_z<4>",
        "/comp_z<3>",
        "/comp_z<2>",
        "/comp_z<1>",
        "/comp_z<0>",
    ],
    logic_threshold=0.5, # Volts
    clock_period=1e-9 # seconds
)
SPECIFICATION.infer_buses() # this lets vcsvgenie recognize <> bus notation

In [3]:
# parameters
thres_samp = 3 # window size for asymptotic analysis regression

In [ ]:
# load and parse vcsv
VCSV_FILE_1 = Path("random30.vcsv")
df1, titles1 = read_vcsv(VCSV_FILE_1)
waveforms1 = vcsvgenie.waveform.construct_waveforms(df1, titles1)
results1 = SPECIFICATION.interpret(waveforms1)
results1.find_transitions()
results1.find_propagations()

# recognize digital data
results1.digitize()
results1.resolve_buses()
bus_data1 = results1.tabulate_bus_data() # Digital data for buses
bus_data1.to_csv("random_30.vcsv")       # users may peruse for verification

In [ ]:
# delay histogram
histogram1 = transient_waveform.delay_histogram(results1.delays, show=True)

In [ ]:
maxtpd_idxes1, maxtpd1 = transient_waveform.find_max_delay_trend(results1.delays)

# If the plot does not level out (converge y-value as x increases),
# this is an indication we haven't tested enough samples
delay_plot1 = transient_waveform.plot_max_delay_trend(maxtpd_idxes1, maxtpd1, show=False)
delay_plot1.suptitle("(Cases=30)")

global_estimation1 = transient_waveform.estimate_global_critical_delay(maxtpd_idxes1, maxtpd1, thres_samp=thres_samp)

# If there is a large difference between these two quantities,
# this is another indication of insufficient samples
print(f"Observed max propagation delay: {maxtpd1[-1]}")
print(f"Estimated global max propagation delay: {global_estimation1}")

In [ ]:
## second example:
# load and parse vcsv
VCSV_FILE_2 = Path("random100.vcsv")
df2, titles2 = read_vcsv(VCSV_FILE_2)
waveforms2 = vcsvgenie.waveform.construct_waveforms(df2, titles2)
results2 = SPECIFICATION.interpret(waveforms2)
results2.find_transitions()
results2.find_propagations()

histogram2 = transient_waveform.delay_histogram(results2.delays, show=True)
maxtpd_idxes2, maxtpd2 = transient_waveform.find_max_delay_trend(results2.delays)
delay_plot2 = transient_waveform.plot_max_delay_trend(maxtpd_idxes2, maxtpd2, show=False)
delay_plot2.suptitle("(Cases=100)")

global_estimation2 = transient_waveform.estimate_global_critical_delay(maxtpd_idxes2, maxtpd2, thres_samp=thres_samp)

print(f"Observed max propagation delay: {maxtpd2[-1]}")
print(f"Estimated global max propagation delay: {global_estimation2}")

In [ ]:
axes: List[Axes] = []
fig, axes = plt.subplots(ncols=1, nrows=2, figsize=(8, 12))
# plt.title("Max delay trends for differently-sized MC simulations")
axes[0].set_title('Max delay trends for differently-sized MC simulations')
axes[0].plot(maxtpd_idxes1, maxtpd1, marker='o', linestyle='--', label='# Test Cases=30', color='r')
axes[1].plot(maxtpd_idxes2, maxtpd2, marker='o', linestyle='--', label='# Test Cases=100', color='b')
for ax in axes:
    ax.set_xlabel('Propagation delay index')
    ax.set_ylabel('Propagation delay (s)')
    ax.grid(visible=True, which='both', axis='both')
    ax.legend(loc='lower right')

# fig.suptitle('Max delay trends for differently-sized MC simulations')
# plt.legend()
plt.show()